# Классификация комментариев на позитивные и негативные
Для интернет магазина необходимо сделать инструмент, который будет определять токсичные комментарии и отправлять их на предмодерацию.

Предоставленный датасет - комментарии с разметкой о токсичности. Столбец *text* содержит сам текст комментария, а *toxic* — пометку о токсичности.

Необходимо:
1. Загрузить и подготовить данные
2. Обучить разные модели
3. Сделать выводы

Ключевая метрика *F1_score*. Необходимо получить качество не меньше 0,75.

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-необходимых-библиотек" data-toc-modified-id="Импорт-необходимых-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт необходимых библиотек</a></span></li><li><span><a href="#Загрузка-и-оценка-данных" data-toc-modified-id="Загрузка-и-оценка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка и оценка данных</a></span></li><li><span><a href="#Лемматизация-текста" data-toc-modified-id="Лемматизация-текста-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация текста</a></span></li></ul></li><li><span><a href="#Модель-BERT" data-toc-modified-id="Модель-BERT-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Модель BERT</a></span><ul class="toc-item"><li><span><a href="#Подготовка-ембендингов" data-toc-modified-id="Подготовка-ембендингов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовка ембендингов</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели</a></span></li></ul></li><li><span><a href="#Модель-&quot;TF-IDF&quot;" data-toc-modified-id="Модель-&quot;TF-IDF&quot;-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Модель "TF-IDF"</a></span><ul class="toc-item"><li><span><a href="#Векторизация-слов" data-toc-modified-id="Векторизация-слов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Векторизация слов</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Обучение модели</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Подготовка данных

### Импорт необходимых библиотек

Установим и импортируем необходимые библиотеки.

In [1]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio===0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip install nltk

In [3]:
!pip install transformers

In [4]:
!pip install lightgbm

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import transformers as ppb
import torch

from sklearn.metrics import f1_score

from tqdm import notebook
import re

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.dummy import DummyClassifier


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

C:\Users\bombe\.conda\envs\practicum\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\bombe\.conda\envs\practicum\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  me

True

### Загрузка и оценка данных
Загрузим предоставленный датасет. Посмотрим первые строки и информацию о нем.

In [7]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
data.info()
print("Количество дубликатов:",data.duplicated().sum())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
Количество дубликатов: 0


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Текст на английском языке. В нем присутствуют спецсимволы и сокращения. Уберем из текста символ переноса строки.

In [8]:
data['text'] = data['text'].apply(lambda x: re.sub(r'[\n]', ' ', x))
data.head()

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,""" More I can't make any real suggestions on im...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Лемматизация текста
Дополнительно лемматизируем текст для использования с мешком слов. 

Оставим в тексте только английские буквы и разделитель `'`. Так же переведем все слова в нижний регистр.

In [9]:
data['text_bag'] = data['text'].apply(lambda x: " ".join(re.sub(r'[^a-zA-z\']', ' ', x).split()).lower())
data.head(10)

,text,toxic,text_bag
0,Explanation Why the edits made under my userna...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he matches this background colour i'm se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i'm really not trying to edit war it's...
3,""" More I can't make any real suggestions on im...",0,more i can't make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...
5,""" Congratulations from me as well, use the to...",0,congratulations from me as well use the tools ...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word 'nonsense' was offensive to ...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


В качестве лемматизатора будем использовать `WordNetLemmatizer` из библиотеки NLTK. Определим лемматизатор и функцию определения POS-тега.

In [10]:
lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

Проверим лемматизатор.

In [11]:
sentence = data.loc[0,'text_bag']
print("До лемматизации:")
print(sentence)
print()
print("После лемматизации:")
print(' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]))

До лемматизации:
explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now

После лемматизации:
explanation why the edits make under my username hardcore metallica fan be revert they be n't vandalism just closure on some gas after i vote at new york doll fac and please do n't remove the template from the talk page since i 'm retire now


Лемматизация работает. `Were` превратилось в `be`, `voted` в `vote`.

Лемматизируем весь корпус. Чтобы при перезапуске ядра не ждать час, сохраним результат в *csv*-файл. Затем будем считывать результат с него.

In [12]:
try:
    data['text_bag'] = pd.read_csv('lemma.csv')['text_bag']
except:
    pack = 1000
    for i in notebook.tqdm(range(data.shape[0]//pack+1)):
        data.loc[i*pack:(i+1)*pack, 'text_bag'] = data.loc[i*pack:(i+1)*pack, 'text_bag'].apply(lambda sentence: ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]))
    data['text_bag'].to_csv('lemma.csv')
data.head()

,text,toxic,text_bag
0,Explanation Why the edits made under my userna...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i 'm really not try to edit war it 's ...
3,""" More I can't make any real suggestions on im...",0,more i ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


На этом подготовка данных закончена.

## Модель BERT
### Подготовка ембендингов

Для обучения моделей подготовим ембендинги фраз при помощи предварительно обученной нейронной сети `DistilBert`. Она быстрее классического `BERT`, при том же качестве работы (или почти том).

Проверим подключение GPU.

In [13]:
torch.cuda.is_available()

True

Подключим GPU для выполнения последующих задач.

In [14]:
device = "cuda:0"
torch.device(device if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

Предварительно необходимо токенезировать корпус текста.

Определим токенизатор и модель.

In [15]:
model_class = ppb.DistilBertModel
tokenizer_class = ppb.DistilBertTokenizer
pretrained_weights = 'distilbert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

model = model_class.from_pretrained(pretrained_weights)
model = model.to(device)

Токенезируем текст.

In [16]:
%%time
tokenized = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))

Wall time: 5min 5s


In [17]:
tokenized[:5]

0    [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1    [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2    [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3    [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4    [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
Name: text, dtype: object

Дополним токенезированный текст нулями до максимальной длины фразы.

In [18]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Создадим маску корпуса. Для добавленных нулей в маске будет стоять 0, для остальных признаков - 1.

In [19]:
attention_mask = np.where(padded != 0, 1.0, 0.0)
attention_mask.shape

(159571, 512)

Выделим ембендинги фраз.
Для уменьшения затра памяти процесс будет идти батчами по 75 фраз за раз.

In [20]:
batch_size = 75
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size+1)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
       
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/2128 [00:00<?, ?it/s]

Данные для моделей готовы.

### Обучение модели

Разделим данные на обучающую и тестовую выборки.

In [35]:
features = np.concatenate(embeddings)
#np.save('bert', features) #защита от повторного прогона берта из-за падения ядра
labels = data['toxic']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels,
                                                                           random_state = 12345, test_size = .25)

Обучим линейную регрессию.

In [79]:
lr_clf = LogisticRegression(class_weight = {0:1,1:2})
lr_clf.fit(train_features, train_labels)
print("f1_score тестовой выборки:", f1_score(test_labels, lr_clf.predict(test_features)))
print("f1_score обучающей выборки:", f1_score(train_labels, lr_clf.predict(train_features)))

f1_score тестовой выборки: 0.7515844414067353
f1_score обучающей выборки: 0.7653131452167928


Обучим случайный лес.

In [33]:
RFC = RandomForestClassifier()
RFC.fit(train_features, train_labels)
print("f1_score тестовой выборки:", f1_score(test_labels, RFC.predict(test_features)))
print("f1_score обучающей выборки:", f1_score(train_labels, RFC.predict(train_features)))

f1_score тестовой выборки: 0.6705957311245621
f1_score обучающей выборки: 0.9688862797718886


Случайный лес со стандартными гиперпараметрами очень сильно переобучается и дает плохой результат.

Попробуем lightgbm.

In [73]:
LGBMC = LGBMClassifier(n_estimators = 1000, random_state=12345, max_depth=8)
LGBMC.fit(train_features, train_labels)
print("f1_score тестовой выборки:", f1_score(test_labels, LGBMC.predict(test_features)))
print("f1_score обучающей выборки:", f1_score(train_labels, LGBMC.predict(train_features)))

f1_score тестовой выборки: 0.7212647770972795
f1_score обучающей выборки: 0.9995925020374897


Результат неудовлетворительный.

## Модель "TF-IDF"
### Векторизация слов

Разделим данные на обучающую и тестовую выборки.

In [80]:
corpus = data['text_bag'].values.astype('U')
labels = data['toxic']

train_features, test_features, train_labels, test_labels = train_test_split(corpus, labels,
                                                                           random_state = 123, test_size = .25)

Векторизируем слова из корпуса при помощи `TfidfVectorizer`. Стоп-слова возьмем из библиотеки NLTK.

In [81]:
stopwords = set(nltk_stopwords.words('english'))
Tfidf = TfidfVectorizer(stop_words=stopwords)
Tfidf.fit(train_features)
train_features = Tfidf.transform(train_features)
test_features = Tfidf.transform(test_features)

### Обучение модели

Обучим линейную регрессию. Воспользуемся функцией `GridSearchCV`.

In [82]:
lr_clf = LogisticRegression()
param_grid = {"C":range(1,15)}

grid_sh = GridSearchCV(lr_clf, param_grid=param_grid, scoring='f1')
grid_sh.fit(train_features, train_labels)
print("Лучший параметр:", grid_sh.best_params_)
print("f1_score тестовой выборки:", f1_score(test_labels, grid_sh.predict(test_features)))
print("f1_score обучающей выборки:", f1_score(train_labels, grid_sh.predict(train_features)))

Лучший параметр: {'C': 10}
f1_score тестовой выборки: 0.7738849022090897
f1_score обучающей выборки: 0.9069126012756422


Не смотря на большое переобучение, результат тестовой выборки очень хороший.

## Вывод

Выводы по моделям получись следующими:
1. Градиентный бустинг для модели `BERT` показал результат 0,72, что хуже линейной регрессии. Возможно это произошло из-за отсутствия перебора гиперпараметров. Для модели `TF-IDF` запустить его не получилось - ломалось ядро. 
2. Модель мешка слов `TF-IDF` показала отличный результат f1, равный 0,77. Для обучения данной модели потребовался 1 час на лемматизицию, все остальное произошло достаточно быстро. В целом на данную модель больших надежд не возлагалось, но она себя оправдала. Можно было бы попробовать со случайным лесом или другими бустинговыми моделями, но я уже побоялся сломать ядро.
3. Ожидалось, что `BERT` вырвется вперед по качеству определения токсичных комментариев. На выделение ембендингов корпуса ему требуется 1,5 часа. Около десяти раз с различным уровнем предобработки (от текста "как есть" до лемматизированного) модель переучивалась. Каждый раз f1_score был в районе 0.75. В работе используется distilBert, но старшая версия BERT, которая работает в 2 раза дольше, показала такой же результат. Ожидаемого грандиозного отрыва не произошло, но минимальный порог для задачи был достигнут.

Проверим, какой результат покажет случайный классификатор.

In [83]:
DC = DummyClassifier(strategy='constant', constant=1)#предсказывает все единицы
DC.fit(train_features, train_labels)
f1_score(test_labels, DC.predict(test_features))

0.18035441421305906

По сравнению со случайной, обе модели работают просто отлично.